# Basic Examples with Different Protocols

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

## Install Seldon Analytics

In [1]:
import os
import json
import time
from pprint import pprint

In [2]:
chart_path = os.path.abspath('../../../helm-charts/seldon-core-analytics')
print(chart_path)

!helm install seldon-core-analytics {chart_path} -n seldon-system --wait

/Users/zyxue/Projects/seldon/seldon-core/helm-charts/seldon-core-analytics
NAME: seldon-core-analytics
LAST DEPLOYED: Tue Feb  9 14:19:39 2021
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1


In [3]:
!kubectl create namespace seldon

namespace/seldon created


In [4]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


## Custom Metrics

In [5]:
!pygmentize model.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metrics:0.2
          name: classifier
          imagePullPolicy: Never
    graph:
      children: []
      name: classifier
      type: MODEL
    name: example
    replicas: 1


In [6]:
!kubectl create -f model.yaml

seldondeployment.machinelearning.seldon.io/seldon-model created


In [7]:
!kubectl rollout status deploy/$(kubectl get deployment -l seldon-deployment-id=seldon-model -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-model-example-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-classifier" successfully rolled out


In [10]:
response_curl=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' \
        -X POST http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/predictions \
        -H "Content-Type: application/json"

In [11]:
assert len(response_curl) == 1
response_curl_json = json.loads(response_curl[0])
assert len(response_curl_json["meta"]["metrics"]) == 4
response_curl_json

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[1.0, 2.0, 5.0]]},
 'meta': {'metrics': [{'key': 'mycounter', 'type': 'COUNTER', 'value': 1},
   {'key': 'mygauge', 'type': 'GAUGE', 'value': 100},
   {'bins': [0, 1, 2],
    'key': 'myhistogram',
    'type': 'HISTOGRAM',
    'value': 0.5},
   {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}],
  'requestPath': {'classifier': 'seldonio/model-with-metrics:0.2'}}}

In [12]:
print("Waiting so metrics can be scraped")
time.sleep(30)

Waiting so metrics can be scraped


In [13]:
%%writefile get-metrics.sh

kubectl run --quiet=true -it --rm curlmetrics --image=tutum/curl --restart=Never -- \
    curl -s seldon-core-analytics-prometheus-seldon.seldon-system/api/v1/query?query=mycounter_total

Overwriting get-metrics.sh


In [14]:
response_get_metrics =! bash get-metrics.sh

In [15]:
assert len(response_get_metrics) == 1
json.loads(response_get_metrics[0])

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'mycounter_total',
     'app': 'seldon-model-example-0-classifier',
     'app_kubernetes_io_managed_by': 'seldon-core',
     'deployment_name': 'seldon-model',
     'fluentd': 'true',
     'image_name': 'seldonio/model-with-metrics',
     'image_version': '0.2',
     'instance': '172.17.0.13:6000',
     'job': 'kubernetes-pods',
     'kubernetes_namespace': 'seldon',
     'kubernetes_pod_name': 'seldon-model-example-0-classifier-6794b688d8-t5nxp',
     'method': 'predict',
     'model_image': 'seldonio/model-with-metrics',
     'model_name': 'classifier',
     'model_version': '0.2',
     'pod_template_hash': '6794b688d8',
     'predictor_name': 'example',
     'predictor_version': 'example',
     'seldon_app': 'seldon-model-example',
     'seldon_app_svc': 'seldon-model-example-classifier',
     'seldon_deployment_id': 'seldon-model',
     'seldon_deployment_name': 'seldon-model',
     'seldon

In [16]:
response_get_metrics_json = json.loads(response_get_metrics[0])
assert response_get_metrics_json["data"]["result"][0]["metric"]["__name__"] == "mycounter_total"
response_get_metrics_json

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'mycounter_total',
     'app': 'seldon-model-example-0-classifier',
     'app_kubernetes_io_managed_by': 'seldon-core',
     'deployment_name': 'seldon-model',
     'fluentd': 'true',
     'image_name': 'seldonio/model-with-metrics',
     'image_version': '0.2',
     'instance': '172.17.0.13:6000',
     'job': 'kubernetes-pods',
     'kubernetes_namespace': 'seldon',
     'kubernetes_pod_name': 'seldon-model-example-0-classifier-6794b688d8-t5nxp',
     'method': 'predict',
     'model_image': 'seldonio/model-with-metrics',
     'model_name': 'classifier',
     'model_version': '0.2',
     'pod_template_hash': '6794b688d8',
     'predictor_name': 'example',
     'predictor_version': 'example',
     'seldon_app': 'seldon-model-example',
     'seldon_app_svc': 'seldon-model-example-classifier',
     'seldon_deployment_id': 'seldon-model',
     'seldon_deployment_name': 'seldon-model',
     'seldon

In [17]:
response_grpccurl = !cd ../../../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}' \
         -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict

In [18]:
response_grpccurl

['{',
 '  "meta": {',
 '    "requestPath": {',
 '      "classifier": "seldonio/model-with-metrics:0.2"',
 '    },',
 '    "metrics": [',
 '      {',
 '        "key": "mycounter",',
 '        "value": 1',
 '      },',
 '      {',
 '        "key": "mygauge",',
 '        "type": "GAUGE",',
 '        "value": 100',
 '      },',
 '      {',
 '        "key": "myhistogram",',
 '        "type": 3,',
 '        "value": 0.5',
 '      },',
 '      {',
 '        "key": "mytimer",',
 '        "type": "TIMER",',
 '        "value": 20.2',
 '      }',
 '    ]',
 '  },',
 '  "data": {',
 '    "names": [',
 '      "t:0",',
 '      "t:1",',
 '      "t:2"',
 '    ],',
 '    "ndarray": [',
 '        [',
 '              1,',
 '              2,',
 '              5',
 '            ]',
 '      ]',
 '  }',
 '}']

In [19]:
response_grpccurl_json = json.loads("".join(response_grpccurl))
assert len(response_grpccurl_json["meta"]["metrics"]) == 4
response_grpccurl_json

{'meta': {'requestPath': {'classifier': 'seldonio/model-with-metrics:0.2'},
  'metrics': [{'key': 'mycounter', 'value': 1},
   {'key': 'mygauge', 'type': 'GAUGE', 'value': 100},
   {'key': 'myhistogram', 'type': 3, 'value': 0.5},
   {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]},
 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[1, 2, 5]]}}

In [20]:
print("Waiting so metrics can be scraped")
time.sleep(30)

Waiting so metrics can be scraped


In [21]:
response_get_metrics_again = !bash get-metrics.sh

In [22]:
response_get_metrics_again_json = json.loads(response_get_metrics_again[0])
assert response_get_metrics_again_json["data"]["result"][0]["metric"]["__name__"] == "mycounter_total"
assert (
    response_get_metrics_again_json["data"]["result"][0]["metric"]["image_name"]
    == "seldonio/model-with-metrics"
)
response_get_metrics_again_json

{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'mycounter_total',
     'app': 'seldon-model-example-0-classifier',
     'app_kubernetes_io_managed_by': 'seldon-core',
     'deployment_name': 'seldon-model',
     'fluentd': 'true',
     'image_name': 'seldonio/model-with-metrics',
     'image_version': '0.2',
     'instance': '172.17.0.13:6000',
     'job': 'kubernetes-pods',
     'kubernetes_namespace': 'seldon',
     'kubernetes_pod_name': 'seldon-model-example-0-classifier-6794b688d8-t5nxp',
     'method': 'predict',
     'model_image': 'seldonio/model-with-metrics',
     'model_name': 'classifier',
     'model_version': '0.2',
     'pod_template_hash': '6794b688d8',
     'predictor_name': 'example',
     'predictor_version': 'example',
     'seldon_app': 'seldon-model-example',
     'seldon_app_svc': 'seldon-model-example-classifier',
     'seldon_deployment_id': 'seldon-model',
     'seldon_deployment_name': 'seldon-model',
     'seldon

In [23]:
!kubectl delete -f model.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


In [24]:
!helm delete seldon-core-analytics --namespace seldon-system

release "seldon-core-analytics" uninstalled
